# Heat Flow Analysis
This notebook performs the heat flow analysis and anomaly quantification for the study area.
It is the main notebook for a regional heat flow analyis.

In [ ]:
import numpy as np
from math import exp
from pyproj import Proj
from plotconfig import *
from cmcrameri.cm import *
from math import log2, ceil
from cache import cached_call
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from zeal2022hf import get_cm_colors
from flottekarte import Map, GeoJSON
from pickle import Pickler, Unpickler
from matplotlib.ticker import LogFormatter
from scipy.spatial.distance import pdist, squareform
from matplotlib.patches import Circle, Rectangle, Polygon as MPolygon
from matplotlib.collections import LineCollection
from reheatfunq import GammaConjugatePrior, AnomalyLS1980, HeatFlowAnomalyPosterior, HeatFlowPredictive
from loaducerf3 import PolygonSelector, Polygon

Configure plots to look good on a HiDPI monitor (you may not need the following configuration if you are not using a HiDPI monitor):

## Data
First we load the data from the previous notebooks.

First the gamma conjugate prior parameters:

In [ ]:
PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V = np.loadtxt("results/05-GCP-Parameters.txt", skiprows=1, delimiter=',')
gcp = GammaConjugatePrior(PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V)

Geometry. If you have adapted the loading of the fault trace (e.g. for another area),
you can replace loading the `merged_surface_trace` from `intermediate/05-SoCal-UCERF3-default-branches.pickle`
with custom code.

In [ ]:
with open('intermediate/02-Geometry.pickle','rb') as f:
    geometry = Unpickler(f).load()
with open('intermediate/05-SoCal-UCERF3-default-branches.pickle','rb') as f:
    ucerf3_res = Unpickler(f).load()
    merged_surface_trace = ucerf3_res["merged_surface_trace"]
    fault_depth = ucerf3_res["d"]

In [ ]:
proj_str = geometry["proj_str"]
selection_polys_xy = geometry["selection_polygons_xy"]
proj = Proj(proj_str)

Heat flow data

In [ ]:
hf_continental_nghf_mW_m2 = np.load('intermediate/heat-flow-selection-mW_m2.npy')
hf_x, hf_y = proj(hf_continental_nghf_mW_m2[1], hf_continental_nghf_mW_m2[2])
hf_xy = np.stack((hf_x,hf_y), axis=1)
hf_mW_m2 = hf_continental_nghf_mW_m2[0]
hf_xyq_mW_m2 = np.stack((hf_x, hf_y, hf_mW_m2), axis=1)

nghf_table_indices = np.loadtxt('results/nghf-selection-indices.csv', delimiter=',', dtype=int)

Coast lines from the [GSHHG](https://www.soest.hawaii.edu/pwessel/gshhg/) (Wessel & Smith, 1996). The GeoJSONs required for the following code to work can be generated from the ESRI shapefile versions of the GSHHG,
for instance using QGIS.

In [ ]:
def limits(xy):
    xlim = np.min(xy[:,0]), np.max(xy[:,0])
    ylim = np.min(xy[:,1]), np.max(xy[:,1])
    Dx = xlim[1] - xlim[0]
    Dy = ylim[1] - ylim[0]
    D = max(Dx,Dy)
    return ((xlim[0]-0.05*D, xlim[1] + 0.05*D),
            (ylim[0]-0.05*D, ylim[1] + 0.05*D))

xlim, ylim = limits(np.concatenate(selection_polys_xy, axis=0))

In [ ]:
coastline = GeoJSON('data/GSHHS_h_1_US.geojson', proj_str, xlim=xlim, ylim=ylim)

#### Data Selection
Here, we show the selection of heat flow data used in the analysis, the fault trace,
and the impact of the fault-generated heat flow anomaly on the heat flow data.

In [ ]:
DMIN = 20e3

In [ ]:
NAMES = ["Mojave", "Carrizo", "Creeping", "North Coast"]

In [ ]:
selectors = [PolygonSelector(Polygon(*poly[:-1,:].T)) for poly in selection_polys_xy]

In [ ]:
xy_ano = [merged_surface_trace[sel.array_mask(merged_surface_trace)] for sel in selectors]
anomalies = [AnomalyLS1980(xya, d) for xya,d in zip(xy_ano,fault_depth)]

In [ ]:
colors = get_cm_colors(vik, 13)
color0 = colors[0]
color1 = colors[8]
color2 = colors[5]
color3 = colors[9]
color4 = colors[2]

First a general map showing the setting and the sub-ROIs:

In [ ]:
LABEL_LOCATIONS = [(-119, 32.9), (-120.85, 34.1), (-122.7,35.1), (-124.1, 37.3)]
LABEL_ANGLES    = [-18,            -29,          -41,          -53]

In [ ]:
fig = plt.figure(figsize=(4.5, 4.5))
#ax_bg = fig.add_axes((0,0,1,1));
ax = fig.add_axes((0.005, 0.005, 0.99, 0.99))
cax= fig.add_axes((0.103, 0.9, 0.25, 0.02))
mp = Map(proj_str, ax, xlim, ylim)
mp.add_data(coastline, facecolor='lightgray')
h0 = ax.plot(*merged_surface_trace.T, color=color0, linewidth=0.9)
mask = (hf_x >= xlim[0]) & (hf_x <= xlim[1]) & (hf_y >= ylim[0]) & (hf_y <= ylim[1])
h = ax.scatter(hf_x[mask], hf_y[mask], c=hf_mW_m2[mask], marker='.', edgecolor='none',
                    cmap=lajolla_r)
fig.colorbar(h, cax=cax, orientation='horizontal')
cax.set_xlabel('Heat flow\n($\\mathrm{mWm}^{-2}$)', ha='right', labelpad=0.2)
for i,poly in enumerate(selection_polys_xy):
    h1 = ax.add_patch(MPolygon(poly, facecolor='none', edgecolor=color4))
    ax.text(*proj(*LABEL_LOCATIONS[i]), NAMES[i], rotation=LABEL_ANGLES[i],
            ha='center', va='center')
ax.legend((h0[0], h1, h),
          ('San Andreas\nfault trace', 'ROIs', 'Heat flow data\nNGHF (filtered)'),
          bbox_to_anchor=(0.14, 0.1, 0.2, 0.1), loc='center')
mp.plot_axes()
fig.savefig('figures/06-ROI-Map.pdf')

Now the heat flow data within the ROIs and the heat flow signatures:

In [ ]:
SHOW_LINKS = False
CI_CONTOURS = [1e-12, 5e-12, 1e-11, 5e-11]

hf_select = []
hf_xy_select = []
ci_select = []
indices_select = []
xy_bounds = []
background_anomaly = []
if SHOW_LINKS:
    neighbors_select = []

for i in range(len(selectors)):
    mask_i = selectors[i].array_mask(hf_xy)
    indices_select.append(nghf_table_indices[mask_i])
    hf_xy_i = hf_xy[mask_i,:]
    hf_i = hf_mW_m2[mask_i]
    ci = anomalies[i](hf_xy_i)

    xmin,ymin = selection_polys_xy[i].min(axis=0)
    xmax,ymax = selection_polys_xy[i].max(axis=0)
    dx = xmax - xmin
    dy = ymax - ymin
    xmin, xmax = xmin - 0.05*dx, xmax + 0.05*dx
    ymin, ymax = ymin - 0.05*dy, ymax + 0.05*dy
    xy_bounds.append((xmin, xmax, ymin, ymax))
    x_bg = np.linspace(xmin, xmax, ceil(300 * dx / (dy+dx)))
    y_bg = np.linspace(ymin, ymax, ceil(301 * dy / (dy+dx)))
    xg,yg = np.meshgrid(x_bg, y_bg)
    background_anomaly.append((x_bg, y_bg, anomalies[i](np.stack((xg.flat, yg.flat),
                                                                 axis=1)).reshape(xg.shape)))
          
    
    hf_select.append(hf_i)
    hf_xy_select.append(hf_xy_i)
    ci_select.append(ci)
    
    if SHOW_LINKS:
        D = squareform(pdist(hf_xy_i))
        too_close = []
        for j in range(hf_xy_i.shape[0]):
            for k in range(j+1, hf_xy_i.shape[0]):
                if D[j,k] < DMIN:
                      too_close.append((j,k))
        neighbors_select.append(too_close)


prefix = ['(a)','(b)','(c)','(d)']
cntrs = []
axes = []
fig = plt.figure(figsize=(6.975,6.975))
for i in range(len(selectors)):
    ax = fig.add_subplot(2,2,i+1)
    axes.append(ax)
    ax.set_aspect('equal')
    ax.set_xlim(xy_bounds[i][:2])
    ax.set_ylim(xy_bounds[i][2:])
    h0 = ax.add_artist(MPolygon(selection_polys_xy[i], facecolor='none', edgecolor=color2))
    cntr = ax.contour(*background_anomaly[i], zorder=1, levels=CI_CONTOURS,
                      linewidths=0.8, colors='gray')
    cntrs.append(cntr)
    if SHOW_LINKS:
        ax.add_collection(LineCollection([(hf_xy_select[i][j], hf_xy_select[i][k])
                                          for j,k in neighbors_select[i]],
                                         linewidth=0.5, color='gray', zorder=0))
    for xy in hf_xy_select[i]:
        h2 = ax.add_patch(Circle(xy, DMIN, facecolor='none', edgecolor='lightgray', zorder=0, linewidth=0.5))
    h = ax.scatter(*hf_xy_select[i].T, c=hf_select[i],
                   edgecolor='none', cmap=batlow)
    h4 = ax.plot(*xy_ano[i].T, color='k', linewidth=1.0)
    if i == 1:
        ax.legend((h0,Line2D([], [], linewidth=0.8, color='gray'), h,
                   Line2D([],[], marker='o', markeredgewidth=0.5, markerfacecolor='none',
                          color='none', markeredgecolor='lightgray', markersize=12),
                   h4[0]),
                  ("ROI", "$c_i$ ($\\mathrm{m}^{-2}$)", "Heat flow\ndata", "Distance\n$d_\\mathrm{min}$",
                   "Fault trace"),
                  framealpha=0.95, fontsize='small')
    ax.set_title(prefix[i] + " " + NAMES[i])
    fig.colorbar(h, label='Heat flow ($\\mathrm{mWm}^{-2}$)', shrink=0.5)

fig.tight_layout()

for i in range(len(selectors)):
    # Fix an issue where inline_spacing depends either on the axis size or
    # the dy/dx ratio:
    ispace = 10 * ((xy_bounds[i][1] - xy_bounds[i][0]) / (xy_bounds[i][3] - xy_bounds[i][2]))**1.5
    axes[i].clabel(cntrs[i], fmt='%1.1e', use_clabeltext=True,
                   inline_spacing=ispace)

fig.savefig('figures/06-Settings.pdf')

Save the data for further analysis:

In [ ]:
import json
with open('results/Mojave-data.json','w') as f:
    json.dump({
        "data_hf" : [hf for hf in hf_select[0]],
        "data_xy" : [tuple(xy) for xy in hf_xy_select[0]],
        "indices" : [int(i) for i in indices_select[0]],
        "ano_xy"  : [tuple(xy) for xy in xy_ano[0]],
        "ano_d"   : float(fault_depth[0]),
        "dmin"    : float(DMIN)
    }, f, sort_keys=True, indent=2)

## Heat Flow Distribution
First, evaluate the posterior predictive distributions within the regions:

In [ ]:
hfp = [HeatFlowPredictive(hf_i, *hf_xy_i.T, gcp, dmin=DMIN, n_bootstrap=1000)
       for hf_i, hf_xy_i in zip(hf_select, hf_xy_select)]

In [ ]:
q = np.linspace(0, 150, 200)

In [ ]:
CDFs = [hfp[i].cdf(q) for i in range(len(hfp))]
PDFs = [hfp[i].pdf(q) for i in range(len(hfp))]

In [ ]:
fig = plt.figure(figsize=(6.975,7.3), dpi=300)

ax_bg = fig.add_axes((0,0,1,1))
ax_bg.set_axis_off()
axes_y = 0.07 + np.arange(len(hfp))[::-1] * 0.243
dy = 0.17
for i in range(len(hfp)):
    y = CDFs[i]
    y2 = PDFs[i]
    hf_i = hf_select[i]
    hf_xy_i = hf_xy_select[i]

    ax_bg.text(0.5, axes_y[i] + dy + 0.005, NAMES[i], ha='center', va='bottom',
               fontsize='large')
    ax = fig.add_axes((0.07, axes_y[i], 0.42, dy))
    ax.set_ylabel('CDF')
    ax.set_ylim(0,1)
    if i == len(hfp)-1:
        ax.set_xlabel('Heat flow ($\\mathrm{mWm}^{-2}$)')
    h0 = ax.plot(q, y, color='k', linestyle='--', linewidth=1.0)
    h1 = ax.step([0]+list(sorted(hf_i)), [0]+list((np.arange(hf_i.size)+1) / hf_i.size), where='post',
                 color=color0, linewidth=1.0)
    for w,ids in hfp[i].bootstrap:
        h2 = ax.step([0]+list(sorted(hf_i[ids])), [0]+list((np.arange(ids.size)+1) / ids.size),
                     where='post', color=color1, zorder=0, rasterized=True, linewidth=0.5)
    if i == 0:
        ax.legend((h0[0],h1[0],h2[0]), ("Posterior\npredictive CDF", "eCDF\n(all data)",
                                        "eCDFs\n($d_\\mathrm{min}$ enforced)"),
                  fontsize='small', loc='lower right')

    ax.text(0.0, 0.9, '(' + chr(ord('a')+2*i) + ')', ha='center', va='center')


    ax = fig.add_axes((0.51, axes_y[i], 0.41, dy))
    if i == len(hfp)-1:
        ax.set_xlabel('Heat flow ($\\mathrm{mWm}^{-2}$)')
    h0 = ax.hist(hf_i, zorder=0, density=True, color=color2, histtype='stepfilled')
    h1 = ax.plot(q, y2, color='k', linewidth=1, linestyle='--')
    ax.set_ylabel('Density ($\\mathrm{m}^2\\mathrm{mW}^{-1}$)')
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position('right')
    for lp,s,n,v in zip(hfp[i].lp, hfp[i].s, hfp[i].n, hfp[i].v):
        h2 = ax.plot(q, GammaConjugatePrior(exp(lp), s, n, v).posterior_predictive(q),
                     color=color3, zorder=0, linewidth=0.5, rasterized=True)

    if i == 0:
        ax.legend((Rectangle((0,0),1,1,color=color2),h1[0],h2[0]),
                  ("All data", "Posterior\npredictive PDF",
                   "Post. pr. PDFs\n($d_\\mathrm{min}$ enforced)"),
                  fontsize='small')

    ax.set_ylim(ax.get_ylim())
    ax.text(0.0, 0.9*ax.get_ylim()[1], '(' + chr(ord('a')+2*i+1) + ')', ha='center', va='center')


fig.savefig(f'figures/06-Posterior-Predictive.pdf')


##### Data inspection
Potentially, investigate some of the distributions in detail here:

In [ ]:
hfp[2].bootstrap

In [ ]:
hfp[2].q

## Heat Flow Anomaly Analysis

In [ ]:
hfap = [HeatFlowAnomalyPosterior(hf_i, *hf_xy_i.T, anomaly, gcp, dmin=DMIN, n_bootstrap=100)
        for hf_i, hf_xy_i, anomaly in zip(hf_select, hf_xy_select, anomalies)]

In [ ]:
P_H = [np.linspace(0, hfap[i].PHmax, 200) for i in range(len(hfap))]

In [ ]:
[hf.PHmax for hf in hfap]

In [ ]:
y  = [hfap[i].pdf(P_H[i]) for i in range(len(hfap))]
y2 = [hfap[i].cdf(P_H[i]) for i in range(len(hfap))]
y3 = [hfap[i].tail(P_H[i]) for i in range(len(hfap))]

In [ ]:
fig = plt.figure(figsize=(6.975,4.5))
#ax_bg = fig.add_axes((0,0,1,1)) # Design canvas
for i in range(len(hfap)):
    ax = fig.add_subplot(2,2,i+1)
    ax.set_title( '(' + chr(ord('a')+i) + ') ' + NAMES[i])
    ax.set_xlabel('Power $P_H$ (MW)')
    ax.set_ylabel('PDF ($\\mathrm{W}^{-1}$)')
    h0 = ax.plot(1e-6*P_H[i], y[i], color=color3, linewidth=1.0)
    ax.set_xlim(0, 1e-6*hfap[i].PHmax)
    ax.set_ylim(0, ax.get_ylim()[1])
    twax = ax.twinx()
    twax.set_ylabel('CDF')
    h1 = twax.plot(1e-6*P_H[i], y2[i], color=color0, linestyle=':', linewidth=1.0)
    h2 = twax.plot(1e-6*P_H[i], y3[i], color='k', linestyle='--', linewidth=1.0)
    twax.set_ylim(0,1)

    if i == 0:
        ax.legend((h0[0],h1[0],h2[0]), ("PDF", "CDF", "Tail distribution"))

fig.tight_layout()
fig.savefig('figures/06-P_H-Posteriors.pdf')

### License
```
The main REHEATFUNQ analysis notebook: compute posterior predictive
distribution and the heat flow anomaly strength posterior distribution.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```